In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
driver = webdriver.Edge()

In [3]:
results=[]

In [4]:
def url_content(url):
    driver.get(url)
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup

In [5]:
def get_product_url(soup):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    product_link= []
    start_link = "https://www.flipkart.com"
    for item in soup.find_all('a',href=True, attrs={'class':'_1fQZEK'}):
        rest_link = item['href']
        product_link.append(start_link+rest_link)
    return product_link


In [6]:
def get_product_name(soup):
    product_name= []
    try:
        name=soup.find('span',attrs={'class':"B_NuCI"}).text
        product_name.append(name)
    except:
        name = "No product name available"
        product_name.append(name)
    #product_name = "Product Name"
     
    return product_name


In [7]:
def get_product_price(soup):
    product_price = []
    try:
        price = soup.find('div',attrs={'class':"_30jeq3 _16Jk6d"}).text
        product_price.append(price)
    except:
        price = "NO price available"
        product_price.append(price)
    
    return product_price


In [8]:
def get_product_ratings(soup):
    product_ratings = []
    try:
        ratings = soup.find('div', attrs = {'class' : '_3LWZlK'}).text
        product_ratings.append(ratings)
    except:
        ratings = "No rating available"
        product_ratings.append(ratings)
    
    return product_ratings


In [9]:
def get_specifications(soup):
    for item in soup.find_all("div", class_="_2418kt"):
            item_specifications = [spec.text for spec in item.find_all("li", class_="_21Ahn-")]
            specifications.append(item_specifications)
    supported_apps = [spec[0].split(': ')[1] for spec in specifications]
    operating_system = [spec[1] for spec in specifications]
    resolution = [spec[2] for spec in specifications]
    sound_output = [spec[3] for spec in specifications]
    refresh_rate = [spec[4] for spec in specifications]
    return (supported_apps,operating_system,resolution,sound_output,refresh_rate)

In [10]:
url = "https://www.flipkart.com/search?q=smart+tv&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_ps&as-pos=1&as-type=HISTORY&suggestionId=smart+tv%7CTelevisions&requestId=ac4665a8-c1ef-462b-9fb8-e5913908cc39"
#driver.get(url)

In [22]:
data = pd.DataFrame({"Product_Url" : [], "Product Name" : [], "Price" : [], "Ratings" : []})

In [23]:
url_contents=url_content(url)

In [24]:
product_link = get_product_url(url_contents)

In [25]:
data["Product_Url"] = product_link

In [26]:
#def assign_to_dataframe(product_content):
    #product_names = get_product_name(product_content)
    #product_prices = get_product_price(product_content)
    #product_ratings= get_product_ratings(product_content)
    #df =pd.DataFrame({
     #   "Product Name": product_names,
      #  "Price": product_prices,
       # "Ratings" : product_ratings
    #})

    #return df

#df = pd.DataFrame()
#for product in range(len(product_link)):
 #   product_url =product_link[product]
  #  product_content = url_content(product_url)
   # df = pd.concat([df, assign_to_dataframe(product_content)], ignore_index=True)
    #df

In [31]:
df = pd.DataFrame()
for product in range(len(data)):
    product_url = data["Product_Url"].iloc[product]
    product_content = url_content(product_url)
    df1 = assign_to_dataframe(product_content)
    df = pd.concat([df, df1], axis=0)
df

,Product Name,Price,Ratings
0,SAMSUNG 80 cm (32 Inch) HD Ready LED Smart Tiz...,"₹12,490",4.4
0,OnePlus Y1 80 cm (32 inch) HD Ready LED Smart ...,"₹12,999",4.3
0,realme 80 cm (32 inch) HD Ready LED Smart Andr...,"₹12,999",4.3
0,Mi 5A 80 cm (32 inch) HD Ready LED Smart Andro...,"₹12,999",4.4
0,OnePlus Y1 100 cm (40 inch) Full HD LED Smart ...,"₹18,999",4.3
0,OnePlus Y1S 80 cm (32 inch) HD Ready LED Smart...,"₹13,999",4.3
0,OnePlus Y1S 108 cm (43 inch) Full HD LED Smart...,"₹23,999",4.3
0,Thomson 139 cm (55 inch) QLED Ultra HD (4K) Sm...,"₹37,999",4.6
0,Infinix Y1 80 cm (32 inch) HD Ready LED Smart ...,"₹8,459",4.2
0,Thomson Alpha 80 cm (32 inch) HD Ready LED Sma...,"₹7,999",4.5


In [28]:
#for product in range(len(data)):
    #product_url =data["Product_Url"].iloc[product]
    #product_content = url_content(product_url)
    #a=get_product_name(product_content)
    #b=get_product_price(product_content)
    #c=get_product_ratings(product_content)
    #get_specifications(product_content)
    #df = pd.DataFrame({
    #"Product Name": a,
    #"Price": b,
    #"Ratings": c
#})
    #def assign_to_dataframe(product_content):
     #   product_names = get_product_name(product_content)
      #  product_prices = get_product_price(product_content)
       # product_ratings= get_product_ratings(product_content)
        #df ={
         #   "Product Name": product_names,
          #  "Price": product_prices,
           # "Ratings" : product_ratings
        #}

        #return df
    #df = assign_to_dataframe(product_content)
    #df1 = pd.DataFrame(df)
    #df1

In [29]:
df = assign_to_dataframe(product_content)
df

,Product Name,Price,Ratings
0,Thomson 164 cm (65 inch) QLED Ultra HD (4K) Sm...,"₹56,999",4.6


In [30]:
#data=df = pd.DataFrame({
 #   "Product_Url" : product_link
 #   "Product Name": get_product_name(soup),
 #   "Price": get_product_price(soup),
   # "Ratings": get_product_ratings(soup)
#})